### 1. mon_feature.pkl upload

In [ ]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = '/content/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'num_outgoing_packets',
    'avg_incoming_burst_size', 'cumul_max',
    'avg_incoming_order_first_30', 'outgoing_first_30'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
18995                    43.91               0.143962   
18996                    15.60               0.019465   
18997                    14.93               0.016411   
18998                    19.91               0.033281   
18999                    13.76               0.011074   

       avg_outgoing_burst_size  num_outgoing_packets  avg_incoming_burst_size  \
0                     1.551282                 121.0                16.666667   
1                     1.702128                  80.0                 9.319149   
2                     1.552632                 118.0                16.3

### 2. SVM

Ignore ConvergenceWarning

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

Split the dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Map all points to have mean=0 and std=1

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train.values)
X_test_scale = scaler.transform(X_test.values)
X_scale = scaler.fit_transform(X.values)

Train and test SVM using RBF kernel

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

clf_rbf = SVC(kernel='rbf', C=100, gamma=0.1, class_weight='balanced')
clf_rbf.fit(X_train_scale, y_train)
y_pred_rbf = clf_rbf.predict(X_test_scale)

print("============ Before Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_rbf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rbf))

============ Before Hyperparameter tuning ============
SVM Accuracy: 0.7597894736842106
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.63      0.66        51
           1       0.86      0.81      0.84        47
           2       0.73      0.92      0.81        48
           3       0.76      0.86      0.81        37
           4       0.72      0.77      0.75        44
           5       0.81      0.75      0.78        56
           6       0.90      0.93      0.92        58
           7       0.59      0.64      0.61        53
           8       0.72      0.63      0.67        49
           9       0.46      0.63      0.53        38
          10       0.81      0.76      0.79        55
          11       0.80      0.80      0.80        45
          12       0.86      0.89      0.88        55
          13       0.54      0.44      0.49        45
          14       0.63      0.69      0.66        52
          15       0.71

### 3. Hyperparameter tuning by using Grid Search

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

search = HalvingGridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train_scale, y_train)

print(search.best_params_)
print(search.best_estimator_)

============ After Hyperparameter tuning ============
SVM Accuracy: 0.7597894736842106
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.63      0.66        51
           1       0.86      0.81      0.84        47
           2       0.73      0.92      0.81        48
           3       0.76      0.86      0.81        37
           4       0.72      0.77      0.75        44
           5       0.81      0.75      0.78        56
           6       0.90      0.93      0.92        58
           7       0.59      0.64      0.61        53
           8       0.72      0.63      0.67        49
           9       0.46      0.63      0.53        38
          10       0.81      0.76      0.79        55
          11       0.80      0.80      0.80        45
          12       0.86      0.89      0.88        55
          13       0.54      0.44      0.49        45
          14       0.63      0.69      0.66        52
          15       0.71 

In [ ]:
y_pred_tuned = search.predict(X_test_scale)

print("============ After Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))